In [7]:
import cv2
import numpy as np
import math


vehicle_net = cv2.dnn.readNet(
    r'C:\Users\Harin\visual studio\project\yolov4.weights',
    r'C:\Users\Harin\visual studio\project\yolov4.cfg'
)
with open(r'C:\Users\Harin\visual studio\project\coco.names', 'r') as f:
    vehicle_classes = f.read().strip().split('\n')

vehicle_layer_names = vehicle_net.getLayerNames()
vehicle_output_layers = [vehicle_layer_names[i - 1] for i in vehicle_net.getUnconnectedOutLayers()]

plate_net = cv2.dnn.readNet(
    r'D:\darknet\backup\yolov4_plate_final.weights',
    r'D:\darknet\data\yolov4_plate.cfg'
)
with open(r'D:\darknet\data\obj.names.txt', 'r') as f:
    plate_classes = [line.strip() for line in f.readlines()]

plate_layer_names = plate_net.getLayerNames()
plate_output_layers = [plate_layer_names[i - 1] for i in plate_net.getUnconnectedOutLayers().flatten()]

class Tracker:
    def __init__(self, start_line=400, end_line=600, distance=20):
        self.center_points = {}
        self.entry_times = {}
        self.speeds = {}
        self.disappeared = {}
        self.id_count = 0
        self.start_line = start_line
        self.end_line = end_line
        self.distance = distance

    def update(self, boxes, video_time):
        objects_bbs_ids = []
        new_center_points = {}
        assigned_ids = set()

        for rect in boxes:
            x, y, w, h = rect
            cx, cy = (x + x + w) // 2, (y + y + h) // 2

            if cy < self.start_line or cy > self.end_line:
                continue

            closest_id = None
            min_dist = float('inf')

            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])
                if dist < 60 and id not in assigned_ids and dist < min_dist:
                    closest_id = id
                    min_dist = dist

            if closest_id is not None:
                self.center_points[closest_id] = (cx, cy)
                new_center_points[closest_id] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, closest_id])
                self.disappeared[closest_id] = 0
                assigned_ids.add(closest_id)

                # Entry time
                if self.start_line - 10 <= cy <= self.start_line + 10 and closest_id not in self.entry_times:
                    self.entry_times[closest_id] = video_time
                    print(f"Vehicle {closest_id} ENTERED at {video_time:.2f} sec")

                # Exit time
                if self.end_line - 10  <= cy <= self.end_line + 10 and closest_id in self.entry_times:
                    exit_time = video_time
                    time_taken = exit_time - self.entry_times[closest_id]

                    if time_taken > 0:
                        speed = (self.distance / time_taken) * 3.6
                        self.speeds[closest_id] = speed
                        print(f"Vehicle {closest_id} EXITED at {exit_time:.2f} sec | Speed: {speed:.2f} km/h")
                    del self.entry_times[closest_id]

            else:
                self.center_points[self.id_count] = (cx, cy)
                new_center_points[self.id_count] = (cx, cy)
                self.disappeared[self.id_count] = 0
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                assigned_ids.add(self.id_count)
                print(f"New Vehicle {self.id_count} detected at ({cx}, {cy})")
                self.id_count += 1

        for id in list(self.center_points.keys()):
            if id not in new_center_points:
                self.disappeared[id] += 1
                if self.disappeared[id] > 10:
                    print(f"Vehicle {id} lost tracking")
                    self.center_points.pop(id)
                    self.disappeared.pop(id)
                    self.speeds.pop(id, None)

        self.center_points = new_center_points.copy()
        return objects_bbs_ids

cap = cv2.VideoCapture(r"C:\Users\Harin\EOC_MFC_speed_detection\speed_vehicles.mp4")
width = int(cap.get(3))
height = int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(r"C:\Users\Harin\EOC_MFC_speed_detection\output_combined_2.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

tracker = Tracker()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    video_time = frame_number / fps

    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    vehicle_net.setInput(blob)
    outputs = vehicle_net.forward(vehicle_output_layers)

    boxes, confidences, class_ids = [], [], []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id in [2, 3, 5, 7]:  # Car, motorcycle, bus, truck
                center_x, center_y, w, h = (detection[:4] * np.array([width, height, width, height])).astype("int")
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))

    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    filtered_boxes = [boxes[i] for i in indices.flatten()] if len(indices) > 0 else []

    tracked = tracker.update(filtered_boxes, video_time)

    for x, y, w, h, obj_id in tracked:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {obj_id}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        if obj_id in tracker.speeds:
            speed_text = f"{tracker.speeds[obj_id]:.2f} km/h"
            cv2.putText(frame, speed_text, (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
    persistent_plates = []

    plate_blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    plate_net.setInput(plate_blob)
    plate_outputs = plate_net.forward(plate_output_layers)

    for output in plate_outputs:
        for det in output:
            scores = det[5:]
            plate_id = np.argmax(scores)
            confidence = scores[plate_id]
            if confidence > 0.5:
                center_x, center_y, w, h = (det[:4] * np.array([width, height, width, height])).astype("int")
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                persistent_plates.append((x, y, w, h))

    for x, y, w, h in persistent_plates:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 0), 2)
        cv2.putText(frame, "Plate", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

    # Draw speed lines
    cv2.line(frame, (0, tracker.start_line), (width, tracker.start_line), (0, 0, 255), 2)
    cv2.putText(frame, "Start", (10, tracker.start_line - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    cv2.line(frame, (0, tracker.end_line), (width, tracker.end_line), (255, 0, 0), 2)
    cv2.putText(frame, "End", (10, tracker.end_line - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    out.write(frame)
    cv2.imshow("Combined Detection", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("✅ All Done! Check output video for results.")



New Vehicle 0 detected at (841, 400)
Vehicle 0 ENTERED at 0.83 sec
New Vehicle 1 detected at (1082, 401)
New Vehicle 2 detected at (1071, 404)
Vehicle 2 ENTERED at 1.70 sec
New Vehicle 3 detected at (1224, 400)
Vehicle 3 ENTERED at 4.23 sec
New Vehicle 4 detected at (1095, 405)
Vehicle 4 ENTERED at 5.00 sec
New Vehicle 5 detected at (1216, 401)
New Vehicle 6 detected at (971, 407)
New Vehicle 7 detected at (1074, 406)
Vehicle 5 ENTERED at 5.23 sec
Vehicle 6 ENTERED at 5.27 sec
New Vehicle 8 detected at (1207, 422)
New Vehicle 9 detected at (954, 404)
New Vehicle 10 detected at (1056, 425)
Vehicle 3 EXITED at 5.47 sec | Speed: 58.38 km/h
New Vehicle 11 detected at (946, 418)
New Vehicle 12 detected at (1100, 404)
Vehicle 12 ENTERED at 7.47 sec
✅ All Done! Check output video for results.
